In [104]:
import random

In [105]:
class GrilleMotsFlechés:
    def __init__(self, lignes, colonnes, grid_init):
        self.lignes = lignes
        self.colonnes = colonnes
        self.grid = [[[None, None] for _ in range(colonnes)] for _ in range(lignes)]                                 #[Nod(a,...), Nod(a,...)] les arbres corrsepondent au moins croisant cette case
        self.grid_init = grid_init

    def draw(self):
        for i in range(self.lignes):
            for j in range(self.colonnes):
                if(self.grid[i][j][0] != None):
                    print(self.grid[i][j][0].e + '|', end='')
                elif(self.grid_init[i][j] == 1):
                    print('¤|', end = '')
                else:
                    print('0|', end = '')
            print()
        
        print()
        print()
  
    
    '''def add_def(self, definition, ligne, colonne):
        if self.grid[ligne][colonne] == '':
            self.grid[ligne][colonne] = definition
        
        else:
            self.grid[ligne][colonne].texte.append(definition.texte[0])
            self.grid[ligne][colonne].direction.append(definition.direction[0])'''

    def add_letter(self, pos, letter):
        #on met à jour les arbres de la case 
        self.grid[pos[0]][pos[1]][0] = self.grid[pos[0]][pos[1]][0].sons[letter]     
        self.grid[pos[0]][pos[1]][1] = self.grid[pos[0]][pos[1]][1].sons[letter]

        #on met à jours les arbres des cases suivantes (horizontalement et verticalement) si nécessaire

        for i in range(pos[0] + 1, self.lignes):
            if(self.grid_init[i][pos[1]] == 0):
                self.grid[i][pos[1]][1] = self.grid[pos[0]][pos[1]][1]
                break
            if(self.grid_init[i][pos[1]] == 1):
                break
        
        for j in range(pos[1] + 1, self.colonnes):
            if(self.grid_init[pos[0]][j] == 0):
                self.grid[pos[0]][j][0] = self.grid[pos[0]][pos[1]][0]
                
                break
            if(self.grid_init[pos[0]][j] == 1):
                break
            
    
    def remove_letter(self, pos):
        self.grid[pos[0]][pos[1]][0] = self.grid[pos[0]][pos[1]][0].father
        self.grid[pos[0]][pos[1]][1] = self.grid[pos[0]][pos[1]][1].father
    
    def next_pos(self, pos):
        l = pos[0]
        c = pos[1]

        if c+1 < self.colonnes:
            return (l, c+1)
        
        return (l+1, 0)

In [106]:
#e représent un character (ou bien le caractère vide également)
class Node:
    def __init__(self, e, sons, father):
        self.e = e 
        self.sons = sons
        self.father = father

    def add_node(self, e):
        if not e in self.sons:
            self.sons[e] = Node(e, {}, self)
        return self.sons[e]
    
    def add_word(self, word):
        if word == '':
            return True
        subTree = self.add_node(word[0])
        subTree.add_word(word[1:])
    
    def print(self):
        print(self.e)

        for fils in self.sons.values():
            fils.print()
            

        

def dico2tree(ordre, word_list):
    tree = Node('', {}, None)
    for word in word_list :
        if(max(ordre) > len(word)):
            print(word," : ",ordre)
        word_permute = ''.join([word[i] for i in ordre])
        tree.add_word(word_permute)
    return tree





In [107]:
def init_grid(dim_grid, grid_init, word_list_list):             #grid_init : contient les cases noires et les mots déjà remplis, world_list_list[3] contient la liste des mots de 3 lettres
    trees = {}                      #la clef ([4,2,1,3,0]) correspondra à un arbre contenant les mots de 5 lettres commencant par la 5ème lettre puis 3 puis 2 puis 4 puis 1
    mot_fleche = GrilleMotsFlechés(dim_grid[0], dim_grid[1], grid_init)
    grid = mot_fleche.grid

    #on rentre dans chaque case la taille du mot restant à partir de cette case, une pemière fois horizontalement puis verticalment 
    counter_letter = 0
    ordre_preremplie = []
    letter_preremplie = []
    coordonne_preremplie = []
    ordre_aremplir = []
    last_first_blanked_cell = [-1, -1]

    for i in range(dim_grid[0]):
        counter_letter = 0
        ordre_preremplie = []
        letter_preremplie = []
        coordonne_preremplie = []
        ordre_aremplir = []
        last_first_blanked_cell = [-1, -1]

        for j in range(dim_grid[1] + 1 ):                                                           #on parcours toujours une case de plus que l'on considère noire sur le bord
                                                                                                      
            
            if(j == dim_grid[1] or grid_init[i][j] == 1):                                           #on arrive sur une case noire, on calcule l'arbre avec le bon ordre si ce n'est pas déjà fait et on le rentre dans last_first_blanked_cell au bon étage
                if(counter_letter > 0):
                    ordre = tuple(ordre_preremplie + ordre_aremplir)
                    if(not ordre in trees):
                        trees[ordre] = dico2tree(ordre, word_list_list[counter_letter])

                    current_tree = trees[ordre]
                    for k, letter in enumerate(letter_preremplie):                                #on va descendre dans l'arbre avec les cases déjà remplie car inutile pour nous ces vilaines mécantes sorcières....
                        current_tree = current_tree.sons[letter]
                        grid[coordonne_preremplie[k][0]][coordonne_preremplie[k][1]][0] = current_tree
                
                if(last_first_blanked_cell != [-1, -1]):
                    grid[last_first_blanked_cell[0]][last_first_blanked_cell[1]][0] = current_tree

                counter_letter = 0
                ordre_preremplie = []
                letter_preremplie = []
                coordonne_preremplie = []
                ordre_aremplir = []
                last_first_blanked_cell = [-1, -1]
                

            elif(grid_init[i][j] == 0):
                ordre_aremplir.append(counter_letter)

                if(last_first_blanked_cell == [-1, -1]):
                    last_first_blanked_cell = [i,j]

                counter_letter += 1
            
            else:
                ordre_preremplie.append(counter_letter)
                letter_preremplie.append(grid_init[i][j])
                coordonne_preremplie.append((i,j))
                
                counter_letter += 1
            


            
    
    for j in range(dim_grid[1]):
        counter_letter = 0
        ordre_preremplie = []
        letter_preremplie = []
        coordonne_preremplie = []
        ordre_aremplir = []
        last_first_blanked_cell = [-1, -1]

        for i in range(dim_grid[0] + 1):
            
            if(i == dim_grid[0] or grid_init[i][j] == 1):                                           #on arrive sur une case noire, on calcule l'arbre avec le bon ordre si ce n'est pas déjà fait et on le rentre dans last_first_blanked_cell au bon étage
                if(counter_letter > 0):
                    ordre = tuple(ordre_preremplie + ordre_aremplir)
                    if(not ordre in trees):
                        trees[ordre] = dico2tree(ordre, word_list_list[counter_letter])

                    current_tree = trees[ordre]
                    for k, letter in enumerate(letter_preremplie):                                #on va descendre dans l'arbre avec les cases déjà remplie car inutile pour nous ces vilaines mécantes sorcières....
                        current_tree = current_tree.sons[letter]
                        grid[coordonne_preremplie[k][0]][coordonne_preremplie[k][1]][1] = current_tree
                
                if(last_first_blanked_cell != [-1, -1]):
                    grid[last_first_blanked_cell[0]][last_first_blanked_cell[1]][1] = current_tree

                counter_letter = 0
                ordre_preremplie = []
                letter_preremplie = []
                coordonne_preremplie = []
                ordre_aremplir = []
                last_first_blanked_cell = [-1, -1]
                

            elif(grid_init[i][j] == 0):
                ordre_aremplir.append(counter_letter)

                if(last_first_blanked_cell == [-1, -1]):
                    last_first_blanked_cell = [i,j]

                counter_letter += 1            
            else:
                ordre_preremplie.append(counter_letter)
                letter_preremplie.append(grid_init[i][j])
                coordonne_preremplie.append((i,j))
        
                counter_letter += 1


            

    return mot_fleche

In [108]:
def complete_grid(mot_fleche, pos):
    if pos == (mot_fleche.lignes, 0):
        mot_fleche.draw()
        return
    
    next_pos = mot_fleche.next_pos(pos)

    
    tree_h = mot_fleche.grid[pos[0]][pos[1]][0]
    tree_v = mot_fleche.grid[pos[0]][pos[1]][1]

 
    if mot_fleche.grid_init[pos[0]][pos[1]] != 0:                                   #il s'agit d'une case contenant une défintion ou un mot prérempli, on ne la touche pas
        complete_grid(mot_fleche, next_pos)
    
    else:

        #print(pos)
        #print(tree_h, " : ", tree_h.sons)
        #print(tree_v, " : ", tree_v.sons)
        #mot_fleche.draw()
        
        # On va récupérer l'intesection des deux listes de lettres possibles afin de tester toutes les possibilités qui nous sont offertes 
        
        poss_v = set(tree_v.sons.keys())
        poss_h = set(tree_h.sons.keys())

        poss = poss_v & poss_h
        poss_list = list(poss)
        random.shuffle(poss_list)

        for scenario in poss_list:
            mot_fleche.add_letter(pos, scenario)
            complete_grid(mot_fleche, next_pos)
            mot_fleche.remove_letter(pos)



In [109]:
def generate_word_list_list(adresse_fichier):
    with open(adresse_fichier, 'r') as fichier:
        lignes = fichier.readlines()

    word_list = [ligne.strip() for ligne in lignes]

    length_list = [len(word) for word in word_list]

    max_length = max(length_list)

    word_list_list = [[] for _ in range(max_length+1)]

    for i in range(len(word_list)):
        word_list_list[len(word_list[i])].append(word_list[i])

    return word_list_list

In [110]:
def generate(dim_grid, grid_init, adresse_fichier):
    word_list_list = generate_word_list_list(adresse_fichier)
    mot_fleche = init_grid(dim_grid, grid_init, word_list_list)
    print("ca commence à calculer :")
    complete_grid(mot_fleche, (0,0))

In [111]:
dim_grid = 10,10
grid_init = [[0 for _ in range(dim_grid[1])] for _ in range(dim_grid[0])]
adresse_fichier = 'fr.txt'

grid_init[0][0] = 1
grid_init[0][2] = 1
grid_init[0][4] = 1
grid_init[0][6] = 1
grid_init[0][8] = 1

grid_init[2][0] = 1
grid_init[2][6] = 1

grid_init[3][5] = 1

grid_init[4][0] = 1
grid_init[4][7] = 1

grid_init[5][8] = 1

grid_init[6][0] = 1
grid_init[6][4] = 1
grid_init[6][6] = 1

grid_init[7][2] = 1
grid_init[7][3] = 1

grid_init[8][0] = 1

grid_init[0][1] = 'Z'
grid_init[1][1] = 'I'
grid_init[2][1] = 'G'
grid_init[3][1] = 'O'
grid_init[4][1] = 'U'
grid_init[5][1] = 'I'
grid_init[6][1] = 'L'
grid_init[7][1] = 'L'
grid_init[8][1] = 'E'
grid_init[9][1] = 'R'

grid_init[1][0] = 'P'
grid_init[1][1] = 'I'
grid_init[1][2] = 'G'
grid_init[1][3] = 'N'
grid_init[1][4] = 'O'
grid_init[1][5] = 'U'
grid_init[1][6] = 'F'
grid_init[1][7] = 'E'
grid_init[1][8] = 'E'
grid_init[1][9] = 'S'

In [112]:
generate(dim_grid, grid_init, adresse_fichier)

ca commence à calculer :
¤|Z|¤|K|¤|G|¤|C|¤|A|
P|I|G|N|O|U|F|E|E|S|
¤|G|R|E|C|S|¤|T|A|S|
G|O|U|R|E|¤|R|O|U|A|
¤|U|N|I|A|X|E|¤|X|I|
S|I|G|A|N|I|D|E|¤|N|
¤|L|E|S|¤|P|¤|R|A|I|
I|L|¤|¤|C|H|E|R|I|R|
¤|E|N|C|R|O|U|E|R|A|
T|R|A|F|U|S|E|R|A|S|


¤|Z|¤|K|¤|G|¤|C|¤|A|
P|I|G|N|O|U|F|E|E|S|
¤|G|R|E|C|S|¤|T|A|S|
G|O|U|R|E|¤|R|O|U|A|
¤|U|N|I|A|X|E|¤|X|I|
S|I|G|A|N|I|D|E|¤|N|
¤|L|E|S|¤|P|¤|R|A|I|
I|L|¤|¤|C|H|E|R|I|R|
¤|E|N|C|R|O|U|E|R|A|
T|R|A|F|U|S|E|R|A|I|


¤|Z|¤|K|¤|G|¤|C|¤|A|
P|I|G|N|O|U|F|E|E|S|
¤|G|R|E|C|S|¤|T|A|S|
G|O|U|R|E|¤|R|O|U|A|
¤|U|N|I|A|X|E|¤|X|I|
S|I|G|A|N|I|D|E|¤|N|
¤|L|E|S|¤|P|¤|R|A|I|
I|L|¤|¤|C|H|E|R|I|R|
¤|E|N|C|R|O|U|E|R|A|
B|R|U|I|S|S|E|R|A|I|


¤|Z|¤|K|¤|G|¤|C|¤|A|
P|I|G|N|O|U|F|E|E|S|
¤|G|R|E|C|S|¤|T|A|S|
G|O|U|R|E|¤|R|O|U|A|
¤|U|N|I|A|X|E|¤|X|I|
S|I|G|A|N|I|D|E|¤|N|
¤|L|E|S|¤|P|¤|R|A|I|
I|L|¤|¤|C|H|E|R|I|R|
¤|E|N|C|R|O|U|E|R|A|
B|R|U|I|S|S|E|R|A|S|


¤|Z|¤|K|¤|G|¤|C|¤|A|
P|I|G|N|O|U|F|E|E|S|
¤|G|R|E|C|S|¤|T|A|S|
G|O|U|R|E|¤|R|O|U|A|
¤|U|N|I|A|X|E|¤|X|I|
S|I|G|A|N|I|D|E|¤|N|
¤

KeyboardInterrupt: 